In [22]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, Aer
from qiskit.circuit.library import EfficientSU2
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
import numpy as np



In [23]:
num_qubits = 12
qc = QuantumCircuit(num_qubits)
j = 0
pi = np.pi
angles = np.array([0.5, 1, 1.5, 1, 1.5, 1, 0.5, 1, 1.5, 1, 0.5, 1]) * pi


for i in range(num_qubits):
    qc.ry(angles[i], i)

for qubit in range(num_qubits - 1):
    qc.cnot(qubit, qubit+1)

# qc.draw()



In [40]:
qasm_text = """OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
sx q[1];
rz(2*pi) q[1];
sx q[1];
rz(3*pi) q[1];
cx q[2],q[0];
cx q[0],q[2];
cx q[2],q[0];
sx q[2];
rz(5*pi/4) q[2];
sx q[2];
rz(3*pi) q[2];
sx q[0];
rz(2.52611294491941) q[0];
sx q[0];
rz(3*pi) q[0];
cx q[1],q[0];
sx q[0];
rz(3.75707236226018) q[0];
sx q[0];
rz(3*pi) q[0];
cx q[1],q[0];
cx q[0],q[1];
cx q[0],q[2];
sx q[2];
rz(3*pi/4) q[2];
sx q[2];
rz(3*pi) q[2];
cx q[0],q[2];
cx q[2],q[0];
"""

qc = QuantumCircuit.from_qasm_str(qasm_text)
qc.draw()

┌───┐                     ┌───┐  ┌────┐  ┌────────────┐┌────┐»
q_0: ──────┤ X ├────────────■────────┤ X ├──┤ √X ├──┤ Rz(2.5261) ├┤ √X ├»
     ┌────┐└─┬─┘┌────────┐  │  ┌────┐└─┬─┘┌─┴────┴─┐└────────────┘└────┘»
q_1: ┤ √X ├──┼──┤ Rz(2π) ├──┼──┤ √X ├──┼──┤ Rz(3π) ├────────────────────»
     └────┘  │  └────────┘┌─┴─┐└────┘  │  └─┬────┬─┘ ┌──────────┐ ┌────┐»
q_2: ────────■────────────┤ X ├────────■────┤ √X ├───┤ Rz(5π/4) ├─┤ √X ├»
                          └───┘             └────┘   └──────────┘ └────┘»
«     ┌────────┐┌───┐┌────┐┌────────────┐┌────┐┌────────┐┌───┐                »
«q_0: ┤ Rz(3π) ├┤ X ├┤ √X ├┤ Rz(3.7571) ├┤ √X ├┤ Rz(3π) ├┤ X ├──■────■────────»
«     └────────┘└─┬─┘└────┘└────────────┘└────┘└────────┘└─┬─┘┌─┴─┐  │        »
«q_1: ────────────■────────────────────────────────────────■──┤ X ├──┼────────»
«     ┌────────┐                                              └───┘┌─┴─┐┌────┐»
«q_2: ┤ Rz(3π) ├───────────────────────────────────────────────────┤ X ├┤ √X ├»
«     └────────┘                                                   └───┘└────┘»
«                                      ┌───┐
«q_0: ──────────────────────────────■──┤ X ├
«                                   │  └─┬─┘
«q_1: ──────────────────────────────┼────┼──
«     ┌──────────┐┌────┐┌────────┐┌─┴─┐  │  
«q_2: ┤ Rz(3π/4) ├┤ √X ├┤ Rz(3π) ├┤ X ├──■──
«     └──────────┘└────┘└────────┘└───┘

In [41]:
from qiskit import BasicAer, transpile

backend = BasicAer.get_backend('statevector_simulator') # the device to run on
result = backend.run(transpile(qc, backend)).result()
psi  = result.get_statevector(qc)




In [44]:
def to_bin(v):
    coef_bin = []
    large = np.log2(len(v))
    for i in range(len(v)):
        if abs(v[i])> 1e-5:
            py_binario = bin(i)[2:]
            binario = '0'*int((large - len(py_binario)))+py_binario
            coef_bin += [(binario,v[i])] 
    return coef_bin


In [51]:
# np.linalg.norm(psi.real)
# for coso in psi:
#     print(coso.real)

ham = np.array([[ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  2.,  0.,  2.,  0.,  0.,  0.],
       [ 0.,  2., -1.,  0.,  2.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  2.,  2.,  0.],
       [ 0.,  2.,  2.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  2.,  0., -1.,  2.,  0.],
       [ 0.,  0.,  0.,  2.,  0.,  2., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.]])


np.dot(np.dot(psi, ham), psi).real


#to_bin(psi)

3.0

In [24]:

np.where(abs(psi) > 0 )

(array([1365, 1366, 1369, 1370, 1381, 1382, 1385, 1386, 1429, 1430, 1433,
        1434, 1445, 1446, 1449, 1450, 1621, 1622, 1625, 1626, 1637, 1638,
        1641, 1642, 1685, 1686, 1689, 1690, 1701, 1702, 1705, 1706, 2389,
        2390, 2393, 2394, 2405, 2406, 2409, 2410, 2453, 2454, 2457, 2458,
        2469, 2470, 2473, 2474, 2645, 2646, 2649, 2650, 2661, 2662, 2665,
        2666, 2709, 2710, 2713, 2714, 2725, 2726, 2729, 2730], dtype=int64),)

### Pruebas de ansatz


In [12]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter

def create_ansatz_1(num_qubits, layers = 1):
    qc = QuantumCircuit(num_qubits)

    for qubit in range(0, num_qubits - (num_qubits % 2), 2):
        qc.h(qubit)
        qc.cnot(qubit, qubit+1)

    j = 0
    for layer in range(layers):
        for qubit in range(num_qubits):
            qc.ry(Parameter('θ_' + str(j)), qubit)
            j += 1
        for qubit in range(num_qubits - 1):
            qc.cnot(qubit, qubit+1)
    return qc

def create_ansatz_2(num_qubits, layers = 1):
    qc = QuantumCircuit(num_qubits)
    j = 0
    for qubit in range(num_qubits):
            qc.ry(Parameter('θ_' + str(j)), qubit)
            j += 1
    for layer in range(layers):
        for qubit in range(num_qubits - 1):
            qc.cnot(qubit, qubit+1)
        for qubit in range(num_qubits):
            qc.ry(Parameter('θ_' + str(j)), qubit)
            j += 1
       
    return qc

def create_ansatz_3(num_qubits, layers = 1):
    qc = QuantumCircuit(num_qubits)
    j = 0
    for qubit in range(num_qubits):
            qc.ry(Parameter('θ_' + str(j)), qubit)
            j += 1
    for layer in range(layers):
        for qubit in range(0, (num_qubits % 2), 2):
            qc.h(qubit)
            qc.cnot(qubit, qubit+1)
        for qubit in range(num_qubits):
            qc.ry(Parameter('θ_' + str(j)), qubit)
            j += 1
       
    return qc

# ANSATZ 4 ES EL QUE ESTAMOS USANDO EN EL PRIMER CASO
# RY CX
def create_ansatz_4(num_qubits, layers = 1):
    qc = QuantumCircuit(num_qubits)
    j = 0
    for layer in range(layers):
        for qubit in range(num_qubits):
            qc.ry(Parameter('θ_' + str(j)), qubit)
            j += 1
        for qubit in range(num_qubits -1):
            qc.cnot(qubit, qubit+1)
        
    return qc

# Controlled RY
num_qubits = 3
layers = 2

def create_ansatz_cry(num_qubits, layers = 1):
    qc = QuantumCircuit(num_qubits)
    j = 0
    for layer in range(layers):
        qc.ry(Parameter('θ_' + str(j)), 0)
        j += 1

        for qubit in range(1, num_qubits):
            qc.cry(Parameter('θ_' + str(j)), qubit - 1, qubit)
            j += 1
    return qc

def create_ansatz_cry_cnot(num_qubits, layers):
    qc = QuantumCircuit(num_qubits)
    j = 0
    for layer in range(layers):
        qc.ry(Parameter('θ_' + str(j)), 0)
        j += 1

        for qubit in range(1, num_qubits):
            qc.cry(Parameter('θ_' + str(j)), qubit - 1, qubit)
            qc.cnot(qubit, qubit -1)
            j += 1
    
    return qc
# qc = create_ansatz_cry(num_qubits)

ansatz = create_ansatz_cry_cnot(3, 1)
params = np.array([3.14159265, 5.05222589, 1.57079636])

ansatz.draw()
qc = ansatz.bind_parameters(params)
qc.draw()



┌────────────────┐                  ┌───┐                       
q_0: ┤ Ry(3.14159265) ├────────■─────────┤ X ├───────────────────────
     └────────────────┘┌───────┴────────┐└─┬─┘                  ┌───┐
q_1: ──────────────────┤ Ry(5.05222589) ├──■──────────■─────────┤ X ├
                       └────────────────┘     ┌───────┴────────┐└─┬─┘
q_2: ─────────────────────────────────────────┤ Ry(1.57079636) ├──■──
                                              └────────────────┘